In [1]:
import numpy as np
import pandas as pd

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv
from scipy.special import jn_zeros
from scipy.optimize import minimize
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_3D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(x[2], x[1])   
    
    return rt+ndt, (theta1, theta2)

In [3]:
def series_bessel_fpt(t, a=1, sigma=1, nu=0, n=100):
    zeros = np.asarray([float(besseljzero(0, i+1)) for i in range(n)])
    fpt = np.zeros(t.shape)
    
    for i in range(t.shape[0]):
        series = np.sum((zeros**(nu+1)/jv(nu+1, zeros)) * np.exp(-(zeros**2 * sigma**2)/(2*a**2)*t[i]))
        fpt[i] = sigma**2/(2**nu * a**2 * gamma(nu + 1)) * series
        
    return interp1d(t, fpt)

In [4]:
def HSDM_3D_likelihood(prms, RT, Theta, N_series):
    a = prms[0]
    ndt = prms[1]
    mu = np.array([prms[2], prms[3], prms[4]])
    
    tt = np.arange(0.001, max(RT)+0.02, 0.02)
    fpt = series_bessel_fpt(tt, a, sigma=1, nu=(mu.shape[0]-2)/2, n=N_series)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': []}

min_threshold = 0.5
max_threshold = 5

min_ndt = 0.1
max_ndt = 1

min_mu = -3.5
max_mu = 3.5

N_series = 250

In [6]:
for n in range(150):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    recovery_df['mu3_true'].append(mu[2])
    
    RT = []
    Theta = []
    
    print(n+1, end=': ')
    for i in range(250):
        rt, theta = simulate_HSDM_3D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    print('Simulation is done!', end=' / ')
    
    min_ans = minimize(HSDM_3D_likelihood,
                       args=(RT, Theta, N_series), 
                       x0=np.array([np.random.uniform(min_threshold, max_threshold),
                                    np.random.uniform(min_ndt, max_ndt), 
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu)]),
                       method='Nelder-Mead', 
                       bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                               (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    recovery_df['mu3_estimate'].append(min_ans.x[4])
    
    print('Estimation is done!')
    
recovery_df = pd.DataFrame(recovery_df)

1: Simulation is done! / Estimation is done!
2: Simulation is done! / Estimation is done!
3: Simulation is done! / Estimation is done!
4: Simulation is done! / Estimation is done!
5: Simulation is done! / Estimation is done!
6: Simulation is done! / Estimation is done!
7: Simulation is done! / Estimation is done!
8: Simulation is done! / Estimation is done!
9: Simulation is done! / Estimation is done!
10: Simulation is done! / Estimation is done!
11: Simulation is done! / Estimation is done!
12: Simulation is done! / Estimation is done!
13: Simulation is done! / Estimation is done!
14: Simulation is done! / Estimation is done!
15: Simulation is done! / Estimation is done!
16: Simulation is done! / Estimation is done!
17: Simulation is done! / Estimation is done!
18: Simulation is done! / Estimation is done!
19: Simulation is done! / Estimation is done!
20: Simulation is done! / Estimation is done!
21: Simulation is done! / Estimation is done!
22: Simulation is done! / Estimation is don

In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
0,2.204156,1.209235,0.637418,0.647182,1.321604,0.734153,1.824526,0.664152,-2.276716,-2.040552
1,2.056291,4.117148,0.947323,0.943616,0.927158,3.116981,-2.893946,3.193281,-3.072185,2.232960
2,4.799921,1.805800,0.809514,1.000000,-3.465520,-1.546147,1.037826,0.484559,1.210550,0.549841
3,1.916032,1.002785,0.417716,0.419716,2.569512,3.500000,-2.239849,-1.193730,-2.273334,-1.251835
4,2.180269,1.194939,0.737464,0.698464,-3.370943,-1.673204,2.288349,1.128242,0.734184,0.355932
...,...,...,...,...,...,...,...,...,...,...
145,3.737095,1.295803,0.569236,0.998860,2.623140,1.418382,-0.834706,-0.485860,-0.487677,-0.305415
146,1.282993,4.307779,0.786171,0.748560,-1.862166,1.985566,2.644953,3.389681,0.088921,-0.141026
147,2.439529,1.281843,0.791849,0.835849,3.062450,1.715531,-0.843939,-0.499894,0.976759,0.531230
148,0.564133,2.369762,0.152569,0.630809,-2.066589,2.738115,1.694637,-0.975417,0.794910,0.227629


In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
threshold_true,1.000000,-0.097742,-0.008471,0.210477,0.123462,0.004321,-0.032888,-0.131530,0.039558,0.001058
threshold_estimate,-0.097742,1.000000,0.004750,-0.284487,-0.189075,0.044594,-0.012268,0.212106,-0.051254,0.059950
ndt_true,-0.008471,0.004750,1.000000,0.485217,0.101859,0.063886,0.175093,0.129698,-0.140646,0.140132
ndt_estimate,0.210477,-0.284487,0.485217,1.000000,0.105574,0.054244,-0.003827,-0.145944,0.051625,0.057658
mu1_true,0.123462,-0.189075,0.101859,0.105574,1.000000,0.458824,-0.031377,-0.053061,-0.151827,-0.057072
mu1_estimate,0.004321,0.044594,0.063886,0.054244,0.458824,1.000000,-0.131817,-0.071310,-0.138001,-0.070257
mu2_true,-0.032888,-0.012268,0.175093,-0.003827,-0.031377,-0.131817,1.000000,0.416152,0.021457,0.197870
mu2_estimate,-0.131530,0.212106,0.129698,-0.145944,-0.053061,-0.071310,0.416152,1.000000,-0.043614,-0.054875
mu3_true,0.039558,-0.051254,-0.140646,0.051625,-0.151827,-0.138001,0.021457,-0.043614,1.000000,0.406305
mu3_estimate,0.001058,0.059950,0.140132,0.057658,-0.057072,-0.070257,0.197870,-0.054875,0.406305,1.000000


In [9]:
recovery_df.to_csv('Series_3d_recovery_{}.csv'.format(N_series))